In [44]:
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

In [45]:
def wait_for_page_to_load(driver, wait):
    title = driver.title
    
    try:
        wait.until(
            lambda d: d.execute_script("return document.readyState") == "complete"
        )
        
    except:
        print(f"The webpage \"{title}\" did not get fully loaded.\n")
        
    else:
        print(f"The webpage \"{title}\" did get fully loaded.\n")

In [46]:
# Setting Options to bypass the website Automation block.

chrome_options = Options()
chrome_options.add_argument("--disable-http2")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--enable-features=NetworkServiceInProcess")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36"
)

In [47]:
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()

In [48]:
# Explicit Wait
wait = WebDriverWait(driver, 10)

In [49]:
# Accessing the Target Webpage
url = "https://www.99acres.com/"
driver.get(url)
wait_for_page_to_load(driver, wait)

The webpage "India Real Estate Property Site - Buy Sell Rent Properties Portal - 99acres.com" did get fully loaded.



In [50]:
# Identifying and Entering Text to the Search Bar
try:
    search_bar = wait.until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="keyword2"]'))
    )
except:
    print("Timeout while locating Search Bar.\n")
else:
    search_bar.send_keys("Kolkata")
    time.sleep(5)

In [51]:
# Selecting the Valid option from the List
try:
    valid_option = wait.until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="0"]'))
    )
except:
    print("Timeout while locating valid Search Option.\n")
else:
    valid_option.click()
    time.sleep(5)

In [52]:
# Click on Search Button
try:
    search_button = wait.until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="searchform_search_btn"]'))
    )
except:
    print("Timeout while clicking \"Search\" Button.\n")
else:
    search_button.click()
    wait_for_page_to_load(driver, wait)

The webpage "Property in Kolkata - Real Estate in Kolkata" did get fully loaded.



In [53]:
# Adjust the Budget Slider
try:
    slider = wait.until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="budgetLeftFilter_max_node"]'))
    )
except:
    print("Timeout while clicking in Budget Slider Circle")
else:
    actions = ActionChains(driver)
    (
        actions
        .click_and_hold(slider)
        .move_by_offset(-73,0)
        .release()
        .perform()
    )
    time.sleep(5)

In [54]:
# Filter Results

# 1. Verified
verified = wait.until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div/div/div[4]/div[3]/div[1]/div[3]/section/div/div/div/div/div[1]/div/div[3]'))
)
verified.click()
time.sleep(2)

#. 2. Ready To Move
ready_to_move = wait.until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div/div/div[4]/div[3]/div[1]/div[3]/section/div/div/div/div/div[1]/div/div[5]'))
)
ready_to_move.click()
time.sleep(2)

# Moving the slider the the right to unhide the remaining filters.

# For First Click
filter_right_button_1st = wait.until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div/div/div[4]/div[3]/div[1]/div[3]/section/div/div/div/div/div[2]'))
)
filter_right_button_1st.click()
time.sleep(2)

# For Second Click
filter_right_button_2nd = wait.until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div/div/div[4]/div[3]/div[1]/div[3]/section/div/div/div/div/div[3]'))
)
filter_right_button_2nd.click()
time.sleep(2)


#. 3. With Photos
with_photos = wait.until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div/div/div[4]/div[3]/div[1]/div[3]/section/div/div/div/div/div[2]/div/div[6]'))
)
with_photos.click()
time.sleep(2)


#. 4. With Videos
with_videos = wait.until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div/div/div[4]/div[3]/div[1]/div[3]/section/div/div/div/div/div[2]/div/div[7]'))
)
with_videos.click()
time.sleep(5)

In [56]:
# Navigate Pages and Extract Data

data = []
page_count = 0

while True:
	page_count += 1
	try:
		time.sleep(6)
		next_page_button = driver.find_element(By.XPATH, "//a[normalize-space()='Next Page >']")
	except:
		print(f"Timeout because we have navigated all the {page_count} pages.\n")
		break
	else:
		try:
			driver.execute_script("window.scrollBy(0, arguments[0].getBoundingClientRect().top - 100);", next_page_button)
			time.sleep(6)
	
			# Scraping the Data
			rows = driver.find_elements(By.CLASS_NAME, "tupleNew__TupleContent")
			for row in rows:
				
				# Property Name
				try:
					name = row.find_element(By.CLASS_NAME, "tupleNew__headingNrera").text
				except:
					name = np.nan

				# Property Location
				try:
					location = row.find_element(By.CLASS_NAME, "tupleNew__propType").text
				except:
					location = np.nan

				# Property Price
				try:
					price = row.find_element(By.CLASS_NAME, "tupleNew__priceValWrap").text
				except:
					price = np.nan

				# Property Area and BHK
				try:
					elements = row.find_elements(By.CLASS_NAME, "tupleNew__area1Type")
				except:
					area, bhk = [np.nan, np.nan]
				else:
					area, bhk = [ele.text for ele in elements]
					
				property = {
					"name": name,
					"location": location,
					"price": price,
					"area": area,
					"bhk": bhk
				}
    
				data.append(property)


			wait.until(
				EC.element_to_be_clickable((By.XPATH, "//a[normalize-space()='Next Page >']"))
			).click()
			time.sleep(5)
        
		except:
			print("Timeout while clicking on \"Next Page\".\n")

Timeout while clicking on "Next Page".

Timeout while clicking on "Next Page".

Timeout because we have navigated all the 91 pages.



In [57]:
# Scraping Data from the Last Page

rows = driver.find_elements(By.CLASS_NAME, "tupleNew__TupleContent")
for row in rows:
    
	# Property Name
	try:
		name = row.find_element(By.CLASS_NAME, "tupleNew__headingNrera").text
	except:
		name = np.nan

	# Property Location
	try:
		location = row.find_element(By.CLASS_NAME, "tupleNew__propType").text
	except:
		location = np.nan

	# Property Price
	try:
		price = row.find_element(By.CLASS_NAME, "tupleNew__priceValWrap").text
	except:
		price = np.nan

	# Property Area and BHK
	try:
		elements = row.find_elements(By.CLASS_NAME, "tupleNew__area1Type")
	except:
		area, bhk = [np.nan, np.nan]
	else:
		area, bhk = [ele.text for ele in elements]
					
	property = {
		"name": name,
		"location": location,
		"price": price,
		"area": area,
		"bhk": bhk
	}
	data.append(property)

In [58]:
time.sleep(2)
driver.quit()

In [59]:
df = pd.DataFrame(data)
df.to_csv("uncleaned_real_state_data.csv", index=False)

In [60]:
len(df)

1278

In [61]:
df

,name,location,price,area,bhk
0,Jai Hanuman Residency,"3 BHK Flat in Konnagar, Hooghly",₹26.93 Lac,"1,036 sqft",3 BHK
1,Jai Hanuman Residency,"3 BHK Flat in Konnagar, Hooghly",₹26.93 Lac,"1,036 sqft",3 BHK
2,Jai Hanuman Residency,"3 BHK Flat in Konnagar, Hooghly",₹26.93 Lac,"1,036 sqft",3 BHK
3,Swan Court\n3.1,"2 BHK Flat in New Town, Kolkata",₹80 Lac,"1,012 sqft",2 BHK
4,Ambey Eco Valley,"2 BHK Flat in New Town, Kolkata",₹90 Lac,941 sqft,2 BHK
...,...,...,...,...,...
1273,Green view,"3 BHK Flat in Action Area 1, Kolkata",₹1.15 Cr,"1,610 sqft",3 BHK
1274,South City\n3.9,"4 BHK Flat in Prince Anwar Shah Rd, Kolkata",₹2.5 Cr,"1,707 sqft",4 BHK
1275,Antrik Apartment,"2 BHK Flat in Sodepur, Kolkata North",₹20 Lac,640 sqft,2 BHK
1276,Natural Haldiram Enclave,"3 BHK Flat in VIP Road, Kolkata",₹90 Lac,"1,669 sqft",3 BHK
